Imports

In [21]:
import warnings
warnings.filterwarnings('ignore')

Extract Atrophy Within Region of Interest

___

## Neuroimaging File Extraction

Here's a breakdown of the inputs:

### 0. Base Directorty (CSF)
- **base_directory**: Absolute path to the base directory containing CSF files.

### 1. Cerebrospinal Fluid (CSF)
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `base_directory` and `glob_name_pattern` fields for each segment. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.

---

Input Directory
- This is the BIDS-style directory which was created by Notebook 02

In [28]:
# Shared Base Directory
base_directory = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd'


This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */*/tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [29]:
shared_glob_path = r'*/*/thresholded_tissue_segment_z_scores'

This is the list of tissue types to use from within each terminal folder. 
- tissue_to_import = ['cerebrospinal_fluid', 'white_matter', 'grey_matter']
- **Leave as unchanged if you are unsure**

In [30]:

tissue_to_import = ['cerebrospinal_fluid', 'white_matter', 'grey_matter']

In [31]:
import os
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
def import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import):
    """
    Imports dataframes based on tissue types from specified directories and glob name patterns.
    
    Parameters:
    - base_directory (str): The base directory where the data resides.
    - shared_glob_path (str): The shared directory path for all tissues.
    - tissue_to_import (list): List of tissues to be imported.
    
    Returns:
    - dict: A dictionary containing dataframes for each specified tissue.
    """

    segments_dict = {}
    for tissue in tissue_to_import:
        glob_path = os.path.join(shared_glob_path, ('*'+tissue+'*'))
        segments_dict[tissue] = glob_path

    dataframes_dict = {}
    nan_handler = {'nan': 0, 'posinf':20, 'neginf':-20}
    for k, v in segments_dict.items():
        dataframes_dict[k] = import_matrices_from_folder(connectivity_path=base_directory, file_pattern=v, convert_nan_to_num=nan_handler)
        print(f'Imported data {k} data with {dataframes_dict[k].shape[0]} voxels and {dataframes_dict[k].shape[1]} patients')
        print(f'Example filename per subject: {dataframes_dict[k].columns[0]}')
        print('\n--------------------------------\n')

    return dataframes_dict


In [32]:
imported_dataframes_by_tissue = import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/*/*/thresholded_tissue_segment_z_scores/*cerebrospinal_fluid*
Imported data cerebrospinal_fluid data with 902629 voxels and 10 patients
Example filename per subject: sub-6_cerebrospinal_fluid_generated_nifti.nii

--------------------------------

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/*/*/thresholded_tissue_segment_z_scores/*white_matter*
Imported data white_matter data with 902629 voxels and 10 patients
Example filename per subject: sub-6_white_matter_generated_nifti.nii

--------------------------------

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/*/*/thresholded_tissue_segment_z_scores/*grey_matter*
Imported data grey_matter data with 902629 voxels and 10 patients
Example filename per subject: sub-6_grey_matter_generated_nifti.nii

--------------------------------



**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

In [33]:
from calvin_utils.file_utils.dataframe_utilities import extract_and_rename_subject_id
def rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id):
    """
    Renames the subjects in the provided dataframes based on the split commands.

    Parameters:
    - dataframes_dict (dict): A dictionary containing dataframes with subjects to be renamed.
    - preceding_id (str): The delimiter for taking the part after the split.
    - proceeding_id (str): The delimiter for taking the part before the split.

    Returns:
    - dict: A dictionary containing dataframes with subjects renamed.
    """

    split_command_dict = {preceding_id: 1, proceeding_id: 0}
    
    for k, v in dataframes_dict.items():
        dataframes_dict[k] = extract_and_rename_subject_id(dataframe=dataframes_dict[k], split_command_dict=split_command_dict)
        print('Dataframe: ', k)
        display(dataframes_dict[k])
        print('------------- \n')

    return dataframes_dict


This Should Often Be Left Default

In [34]:
preceding_id = 'sub-'
proceeding_id = '_'

In [35]:
thresholded_atrophy_df_dict = rename_dataframe_subjects(imported_dataframes_by_tissue, preceding_id, proceeding_id)

Dataframe:  cerebrospinal_fluid


,6,1,8,9,7,11,2,5,4,3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  white_matter


,6,1,8,9,7,11,2,5,4,3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  grey_matter


,6,1,8,9,7,11,2,5,4,3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 



# Extract Region of Interest Atrophy

Import Region of Interest Masks

In [36]:
folder_to_import_from = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois'
shared_glob_pattern = '*'

In [37]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
region_of_interest_df = import_matrices_from_folder(connectivity_path=folder_to_import_from, file_pattern=shared_glob_pattern)
region_of_interest_df

I will search:  /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois/*


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


Extract Damage Scores Per Region of Interest

In [38]:
import pandas as pd
import numpy as np
def calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df, count_voxels=True):
    """
    Calculates damage scores for each region of interest based on thresholded atrophy data.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - region_of_interest_df (DataFrame): DataFrame containing regions of interest.
    
    Returns:
    - dict: A dictionary containing damage scores for each region of interest for each patient.
    """
    
    damage_df_dict = {}
    
    for k, v in thresholded_atrophy_df_dict.items():
        
        # Initialize the Dataframe
        damage_df_dict[k] = pd.DataFrame(index=v.columns, columns=region_of_interest_df.columns)
        
        # Iterate Over Each Region of Interest
        for matrix in damage_df_dict[k].columns:
            # Extract Damage Score for Each Patient
            for subject in damage_df_dict[k].index:
                # Mask the subject dataframe to the matrix at hand
                intersection = v[subject].where(region_of_interest_df[matrix] > 0, 0)
                
                # Use multiplication to zero values outside ROI
                weighted_overlap = intersection * region_of_interest_df[matrix]
                
                # Assess overall damage score & atrophied voxels within ROI
                damage_score = weighted_overlap.sum()
                num_voxels = np.count_nonzero(weighted_overlap)
                
                # Assign values to DF 
                damage_df_dict[k].loc[subject, matrix] = damage_score
                damage_df_dict[k].loc[subject, 'num_atrophied_voxels_'+matrix] = num_voxels
                
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [39]:
damage_df_dict = calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df)


Dataframe:  cerebrospinal_fluid


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,num_atrophied_voxels_mni_Cerebellum.nii.gz,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz
6,1973.78569,22387.099629,1429.141497,5235.334243,25514.608833,200.017285,3092.281991,7965.842202,7026.230726,646.477322,592.0,6401.0,307.0,1436.0,7203.0,1294.0,896.0,2261.0,1973.0,208.0
1,1545.567162,13488.127844,1825.294536,3515.628528,16839.100526,125.429717,2181.896571,4642.886776,3746.908621,152.568294,410.0,3678.0,434.0,785.0,4524.0,674.0,632.0,1356.0,1027.0,35.0
8,1022.014212,32685.275846,5967.629179,5451.984094,39882.869719,183.811271,4071.261641,16759.941881,10436.3289,122.279182,273.0,7571.0,746.0,1338.0,8672.0,1328.0,963.0,3415.0,2293.0,39.0
9,3266.285416,48188.500374,5155.86197,12236.475934,60027.115072,299.766363,6876.334622,20827.146078,11681.781552,1000.712117,987.0,12527.0,1171.0,2835.0,15906.0,3171.0,1720.0,5436.0,3233.0,302.0
7,2268.352865,50245.861671,5207.456481,13908.595513,58069.906305,551.477258,5313.648745,22444.626238,11390.113509,696.507513,616.0,11265.0,881.0,2838.0,12911.0,2465.0,1181.0,4731.0,2878.0,168.0
11,11904.069019,46948.869401,6914.33231,8309.761891,67069.926959,242.822702,15249.790637,12113.922056,14677.325667,334.566178,2277.0,11426.0,1079.0,1671.0,15265.0,2534.0,3206.0,3297.0,3880.0,71.0
2,2599.517669,22275.602356,2405.729921,6983.432748,27773.926888,367.444205,2314.687095,9909.379658,3621.446183,780.827023,409.0,5276.0,356.0,1322.0,6239.0,1087.0,563.0,2405.0,1082.0,182.0
5,2558.567631,27566.633886,4928.805473,8291.861069,36927.635855,899.24041,5308.319599,8507.695637,6728.579536,602.34143,642.0,7513.0,1140.0,2047.0,9940.0,2931.0,1476.0,2273.0,2007.0,174.0
4,1180.462808,54993.166807,10839.10275,9150.3289,70629.765736,246.34982,10562.890229,22034.794471,16480.12925,513.862571,219.0,14413.0,2666.0,2130.0,18707.0,3433.0,2344.0,6123.0,4480.0,173.0
3,3510.208616,12812.206415,2951.34243,3522.359111,19285.100527,57.639139,3838.64728,2305.410767,3966.050529,90.476129,834.0,3423.0,360.0,828.0,4648.0,1020.0,1032.0,603.0,1136.0,13.0


------------- 

Dataframe:  white_matter


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,num_atrophied_voxels_mni_Cerebellum.nii.gz,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz
6,-610.920482,-1331.01154,-1598.945147,-263.352201,-3608.082557,-0.0,-353.922245,-781.480277,-888.124459,-22.364687,219.0,542.0,597.0,100.0,1381.0,250.0,144.0,307.0,354.0,9.0
1,-4254.501484,-3146.375011,-13987.750402,-1278.937776,-21494.765675,-40.011261,-493.658024,-2022.765156,-1027.323102,-113.88249,1507.0,1239.0,4855.0,490.0,7638.0,2697.0,203.0,791.0,392.0,44.0
8,-181.838162,-1617.436439,-2268.35611,-184.822463,-4088.501436,-2.718192,-217.05954,-1694.042164,-736.166996,-14.883073,72.0,639.0,817.0,78.0,1537.0,187.0,91.0,632.0,273.0,6.0
9,-215.179102,-1488.479292,-1590.068783,-577.543516,-3309.457694,-17.810289,-732.578198,-780.432455,-442.132234,-18.397677,83.0,610.0,588.0,223.0,1286.0,278.0,293.0,304.0,179.0,8.0
7,-71.9676,-1429.942974,-2233.800465,-407.271729,-3752.61893,-2.674281,-397.103045,-1301.301327,-358.977022,-9.648852,29.0,553.0,785.0,159.0,1374.0,304.0,155.0,463.0,147.0,4.0
11,-2117.894038,-2590.708846,-7146.935787,-1674.295699,-11978.072154,-6.383634,-344.197282,-1524.902403,-746.312588,-38.395573,624.0,999.0,2218.0,620.0,3880.0,1105.0,133.0,588.0,268.0,14.0
2,-8364.006428,-2233.530946,-7478.199009,-545.129452,-18173.194154,-6.55246,-832.762155,-2242.633617,-580.754874,-48.528816,2706.0,847.0,2060.0,198.0,5643.0,1175.0,306.0,794.0,227.0,16.0
5,-398.564332,-1182.782911,-2434.603417,-224.105414,-4085.669061,-10.362631,-327.566281,-1252.994909,-411.695313,-31.759324,159.0,480.0,929.0,93.0,1594.0,374.0,131.0,493.0,165.0,12.0
4,-23243.05533,-2150.33519,-11114.122589,-819.174219,-36712.648829,-7.828806,-837.672784,-1109.320938,-1040.380653,-30.280645,4829.0,876.0,3308.0,332.0,9080.0,2632.0,336.0,448.0,427.0,12.0
3,-750.918633,-1537.870293,-5109.884452,-367.208708,-7449.924526,-4.878979,-710.403726,-788.900062,-717.816508,-52.316408,280.0,621.0,1688.0,149.0,2607.0,860.0,273.0,315.0,281.0,19.0


------------- 

Dataframe:  grey_matter


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,num_atrophied_voxels_mni_Cerebellum.nii.gz,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz
6,-293.70078,-15371.144963,-1633.504534,-2841.712736,-17418.629775,-26.699037,-1949.391047,-5297.238885,-5699.335558,-146.144345,114.0,5812.0,595.0,1074.0,6572.0,934.0,756.0,2014.0,2135.0,58.0
1,-6168.55733,-21595.015867,-3369.967348,-7031.783502,-31048.117674,-167.49219,-3437.53358,-8073.420813,-3482.283378,-351.225592,2134.0,7815.0,1163.0,2504.0,11109.0,2223.0,1246.0,2951.0,1281.0,126.0
8,-368.494984,-16166.912115,-2740.390568,-3074.57748,-19481.037448,-368.20314,-1515.190544,-9385.483552,-2638.694379,-294.129326,139.0,6045.0,983.0,1185.0,7253.0,1380.0,589.0,3424.0,1010.0,114.0
9,-1085.930651,-16162.564607,-1795.891059,-7485.794101,-19235.695745,-325.434047,-2062.753556,-3918.80053,-2849.629441,-350.72448,407.0,5926.0,657.0,2607.0,7066.0,2063.0,794.0,1488.0,1100.0,131.0
7,-1008.527986,-26068.414192,-8656.147186,-8934.517598,-35894.615531,-507.09826,-3014.00789,-10960.326897,-3241.697425,-803.442326,362.0,9479.0,2642.0,3204.0,12559.0,3123.0,1131.0,3947.0,1238.0,295.0
11,-18353.335729,-22604.058141,-11271.086018,-2751.810052,-50902.123877,-141.052333,-8520.778154,-5332.645851,-7662.688346,-71.174972,5500.0,7896.0,2966.0,965.0,16011.0,2794.0,2965.0,1888.0,2651.0,29.0
2,-44420.488471,-30687.737015,-8722.373643,-7451.727038,-82301.817159,-77.915536,-7948.192454,-10028.836586,-5748.908601,-549.70974,11129.0,10674.0,2461.0,2534.0,23859.0,3540.0,2711.0,3520.0,2109.0,204.0
5,-1035.585174,-24222.360371,-2857.365305,-4219.820334,-28282.88123,-382.436676,-2856.36771,-12322.264379,-5566.970593,-471.006837,401.0,9124.0,1082.0,1643.0,10675.0,1983.0,1060.0,4604.0,2100.0,183.0
4,-44285.073467,-25474.342906,-1437.443428,-4286.093716,-69311.697098,-63.771461,-5375.35279,-9755.93515,-6872.455387,-272.864455,11378.0,9339.0,536.0,1600.0,20725.0,1994.0,1870.0,3598.0,2573.0,108.0
3,-4866.267993,-18764.292562,-3788.333235,-3513.403095,-27390.414654,-140.467294,-4797.430688,-5595.713669,-5484.544584,-325.653698,1741.0,7026.0,1370.0,1315.0,10138.0,1958.0,1769.0,2089.0,2087.0,126.0


------------- 



Extract Total Atrophy Volume

In [40]:
import numpy as np
from nimlab import datasets as nimds
def calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict):
    """
    Calculates the total number of atrophy voxels within a mask and updates the damage_df_dict with this information.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - damage_df_dict (dict): Dictionary containing dataframes to which the 'Total Atrophy Voxels' column will be added.
    
    Returns:
    - dict: The updated damage_df_dict with a new column 'Total Atrophy Voxels'.
    """
    
    # Get the mask and brain indices
    mni_mask = nimds.get_img("mni_icbm152")
    mask_data = mni_mask.get_fdata().flatten()
    brain_indices = np.where(mask_data > 0)[0]
    
    for k, v in thresholded_atrophy_df_dict.items():
        # initialize the column
        damage_df_dict[k]['Total Atrophy Voxels'] = np.nan
        for patient_id in v.columns:
            # Filter the dataframe using brain indices
            filtered_df = v[patient_id].iloc[brain_indices]
            damage_df_dict[k].loc[patient_id, 'Total Atrophy Voxels'] = np.count_nonzero(filtered_df)
        
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [41]:
damage_df_dict = calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict)

Dataframe:  cerebrospinal_fluid


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
6,1973.78569,22387.099629,1429.141497,5235.334243,25514.608833,200.017285,3092.281991,7965.842202,7026.230726,646.477322,...,6401.0,307.0,1436.0,7203.0,1294.0,896.0,2261.0,1973.0,208.0,7146.0
1,1545.567162,13488.127844,1825.294536,3515.628528,16839.100526,125.429717,2181.896571,4642.886776,3746.908621,152.568294,...,3678.0,434.0,785.0,4524.0,674.0,632.0,1356.0,1027.0,35.0,4515.0
8,1022.014212,32685.275846,5967.629179,5451.984094,39882.869719,183.811271,4071.261641,16759.941881,10436.3289,122.279182,...,7571.0,746.0,1338.0,8672.0,1328.0,963.0,3415.0,2293.0,39.0,8664.0
9,3266.285416,48188.500374,5155.86197,12236.475934,60027.115072,299.766363,6876.334622,20827.146078,11681.781552,1000.712117,...,12527.0,1171.0,2835.0,15906.0,3171.0,1720.0,5436.0,3233.0,302.0,15393.0
7,2268.352865,50245.861671,5207.456481,13908.595513,58069.906305,551.477258,5313.648745,22444.626238,11390.113509,696.507513,...,11265.0,881.0,2838.0,12911.0,2465.0,1181.0,4731.0,2878.0,168.0,12870.0
11,11904.069019,46948.869401,6914.33231,8309.761891,67069.926959,242.822702,15249.790637,12113.922056,14677.325667,334.566178,...,11426.0,1079.0,1671.0,15265.0,2534.0,3206.0,3297.0,3880.0,71.0,14991.0
2,2599.517669,22275.602356,2405.729921,6983.432748,27773.926888,367.444205,2314.687095,9909.379658,3621.446183,780.827023,...,5276.0,356.0,1322.0,6239.0,1087.0,563.0,2405.0,1082.0,182.0,6219.0
5,2558.567631,27566.633886,4928.805473,8291.861069,36927.635855,899.24041,5308.319599,8507.695637,6728.579536,602.34143,...,7513.0,1140.0,2047.0,9940.0,2931.0,1476.0,2273.0,2007.0,174.0,9768.0
4,1180.462808,54993.166807,10839.10275,9150.3289,70629.765736,246.34982,10562.890229,22034.794471,16480.12925,513.862571,...,14413.0,2666.0,2130.0,18707.0,3433.0,2344.0,6123.0,4480.0,173.0,17864.0
3,3510.208616,12812.206415,2951.34243,3522.359111,19285.100527,57.639139,3838.64728,2305.410767,3966.050529,90.476129,...,3423.0,360.0,828.0,4648.0,1020.0,1032.0,603.0,1136.0,13.0,4596.0


------------- 

Dataframe:  white_matter


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
6,-610.920482,-1331.01154,-1598.945147,-263.352201,-3608.082557,-0.0,-353.922245,-781.480277,-888.124459,-22.364687,...,542.0,597.0,100.0,1381.0,250.0,144.0,307.0,354.0,9.0,1380.0
1,-4254.501484,-3146.375011,-13987.750402,-1278.937776,-21494.765675,-40.011261,-493.658024,-2022.765156,-1027.323102,-113.88249,...,1239.0,4855.0,490.0,7638.0,2697.0,203.0,791.0,392.0,44.0,7635.0
8,-181.838162,-1617.436439,-2268.35611,-184.822463,-4088.501436,-2.718192,-217.05954,-1694.042164,-736.166996,-14.883073,...,639.0,817.0,78.0,1537.0,187.0,91.0,632.0,273.0,6.0,1535.0
9,-215.179102,-1488.479292,-1590.068783,-577.543516,-3309.457694,-17.810289,-732.578198,-780.432455,-442.132234,-18.397677,...,610.0,588.0,223.0,1286.0,278.0,293.0,304.0,179.0,8.0,1284.0
7,-71.9676,-1429.942974,-2233.800465,-407.271729,-3752.61893,-2.674281,-397.103045,-1301.301327,-358.977022,-9.648852,...,553.0,785.0,159.0,1374.0,304.0,155.0,463.0,147.0,4.0,1373.0
11,-2117.894038,-2590.708846,-7146.935787,-1674.295699,-11978.072154,-6.383634,-344.197282,-1524.902403,-746.312588,-38.395573,...,999.0,2218.0,620.0,3880.0,1105.0,133.0,588.0,268.0,14.0,3876.0
2,-8364.006428,-2233.530946,-7478.199009,-545.129452,-18173.194154,-6.55246,-832.762155,-2242.633617,-580.754874,-48.528816,...,847.0,2060.0,198.0,5643.0,1175.0,306.0,794.0,227.0,16.0,5640.0
5,-398.564332,-1182.782911,-2434.603417,-224.105414,-4085.669061,-10.362631,-327.566281,-1252.994909,-411.695313,-31.759324,...,480.0,929.0,93.0,1594.0,374.0,131.0,493.0,165.0,12.0,1591.0
4,-23243.05533,-2150.33519,-11114.122589,-819.174219,-36712.648829,-7.828806,-837.672784,-1109.320938,-1040.380653,-30.280645,...,876.0,3308.0,332.0,9080.0,2632.0,336.0,448.0,427.0,12.0,9073.0
3,-750.918633,-1537.870293,-5109.884452,-367.208708,-7449.924526,-4.878979,-710.403726,-788.900062,-717.816508,-52.316408,...,621.0,1688.0,149.0,2607.0,860.0,273.0,315.0,281.0,19.0,2606.0


------------- 

Dataframe:  grey_matter


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
6,-293.70078,-15371.144963,-1633.504534,-2841.712736,-17418.629775,-26.699037,-1949.391047,-5297.238885,-5699.335558,-146.144345,...,5812.0,595.0,1074.0,6572.0,934.0,756.0,2014.0,2135.0,58.0,6559.0
1,-6168.55733,-21595.015867,-3369.967348,-7031.783502,-31048.117674,-167.49219,-3437.53358,-8073.420813,-3482.283378,-351.225592,...,7815.0,1163.0,2504.0,11109.0,2223.0,1246.0,2951.0,1281.0,126.0,11095.0
8,-368.494984,-16166.912115,-2740.390568,-3074.57748,-19481.037448,-368.20314,-1515.190544,-9385.483552,-2638.694379,-294.129326,...,6045.0,983.0,1185.0,7253.0,1380.0,589.0,3424.0,1010.0,114.0,7250.0
9,-1085.930651,-16162.564607,-1795.891059,-7485.794101,-19235.695745,-325.434047,-2062.753556,-3918.80053,-2849.629441,-350.72448,...,5926.0,657.0,2607.0,7066.0,2063.0,794.0,1488.0,1100.0,131.0,7056.0
7,-1008.527986,-26068.414192,-8656.147186,-8934.517598,-35894.615531,-507.09826,-3014.00789,-10960.326897,-3241.697425,-803.442326,...,9479.0,2642.0,3204.0,12559.0,3123.0,1131.0,3947.0,1238.0,295.0,12547.0
11,-18353.335729,-22604.058141,-11271.086018,-2751.810052,-50902.123877,-141.052333,-8520.778154,-5332.645851,-7662.688346,-71.174972,...,7896.0,2966.0,965.0,16011.0,2794.0,2965.0,1888.0,2651.0,29.0,15994.0
2,-44420.488471,-30687.737015,-8722.373643,-7451.727038,-82301.817159,-77.915536,-7948.192454,-10028.836586,-5748.908601,-549.70974,...,10674.0,2461.0,2534.0,23859.0,3540.0,2711.0,3520.0,2109.0,204.0,23846.0
5,-1035.585174,-24222.360371,-2857.365305,-4219.820334,-28282.88123,-382.436676,-2856.36771,-12322.264379,-5566.970593,-471.006837,...,9124.0,1082.0,1643.0,10675.0,1983.0,1060.0,4604.0,2100.0,183.0,10671.0
4,-44285.073467,-25474.342906,-1437.443428,-4286.093716,-69311.697098,-63.771461,-5375.35279,-9755.93515,-6872.455387,-272.864455,...,9339.0,536.0,1600.0,20725.0,1994.0,1870.0,3598.0,2573.0,108.0,20720.0
3,-4866.267993,-18764.292562,-3788.333235,-3513.403095,-27390.414654,-140.467294,-4797.430688,-5595.713669,-5484.544584,-325.653698,...,7026.0,1370.0,1315.0,10138.0,1958.0,1769.0,2089.0,2087.0,126.0,10130.0


------------- 



Organize the Subjects

In [42]:
def sort_dataframes_by_index(damage_df_dict):
    """
    Attempts to convert the index of each dataframe in damage_df_dict to integers 
    and then sorts the dataframe by its index in ascending order.
    
    Parameters:
    - damage_df_dict (dict): Dictionary containing dataframes to be sorted.
    
    Returns:
    - dict: The sorted damage_df_dict.
    """
    
    sorted_df_dict = {}
    
    for k, df in damage_df_dict.items():
        try:
            # Convert index to integers
            df.index = df.index.astype(int)
            
            # Sort dataframe by index
            sorted_df = df.sort_index(ascending=True)
            sorted_df_dict[k] = sorted_df
            
            # Display the results
            print('Dataframe: ', k)
            display(sorted_df)
            print('------------- \n')
            
        except ValueError:
            # If conversion to integer fails, just use the original dataframe
            sorted_df_dict[k] = df

    return sorted_df_dict


In [43]:
sorted_damage_df_dict = sort_dataframes_by_index(damage_df_dict)

Dataframe:  cerebrospinal_fluid


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
1,1545.567162,13488.127844,1825.294536,3515.628528,16839.100526,125.429717,2181.896571,4642.886776,3746.908621,152.568294,...,3678.0,434.0,785.0,4524.0,674.0,632.0,1356.0,1027.0,35.0,4515.0
2,2599.517669,22275.602356,2405.729921,6983.432748,27773.926888,367.444205,2314.687095,9909.379658,3621.446183,780.827023,...,5276.0,356.0,1322.0,6239.0,1087.0,563.0,2405.0,1082.0,182.0,6219.0
3,3510.208616,12812.206415,2951.34243,3522.359111,19285.100527,57.639139,3838.64728,2305.410767,3966.050529,90.476129,...,3423.0,360.0,828.0,4648.0,1020.0,1032.0,603.0,1136.0,13.0,4596.0
4,1180.462808,54993.166807,10839.10275,9150.3289,70629.765736,246.34982,10562.890229,22034.794471,16480.12925,513.862571,...,14413.0,2666.0,2130.0,18707.0,3433.0,2344.0,6123.0,4480.0,173.0,17864.0
5,2558.567631,27566.633886,4928.805473,8291.861069,36927.635855,899.24041,5308.319599,8507.695637,6728.579536,602.34143,...,7513.0,1140.0,2047.0,9940.0,2931.0,1476.0,2273.0,2007.0,174.0,9768.0
6,1973.78569,22387.099629,1429.141497,5235.334243,25514.608833,200.017285,3092.281991,7965.842202,7026.230726,646.477322,...,6401.0,307.0,1436.0,7203.0,1294.0,896.0,2261.0,1973.0,208.0,7146.0
7,2268.352865,50245.861671,5207.456481,13908.595513,58069.906305,551.477258,5313.648745,22444.626238,11390.113509,696.507513,...,11265.0,881.0,2838.0,12911.0,2465.0,1181.0,4731.0,2878.0,168.0,12870.0
8,1022.014212,32685.275846,5967.629179,5451.984094,39882.869719,183.811271,4071.261641,16759.941881,10436.3289,122.279182,...,7571.0,746.0,1338.0,8672.0,1328.0,963.0,3415.0,2293.0,39.0,8664.0
9,3266.285416,48188.500374,5155.86197,12236.475934,60027.115072,299.766363,6876.334622,20827.146078,11681.781552,1000.712117,...,12527.0,1171.0,2835.0,15906.0,3171.0,1720.0,5436.0,3233.0,302.0,15393.0
11,11904.069019,46948.869401,6914.33231,8309.761891,67069.926959,242.822702,15249.790637,12113.922056,14677.325667,334.566178,...,11426.0,1079.0,1671.0,15265.0,2534.0,3206.0,3297.0,3880.0,71.0,14991.0


------------- 

Dataframe:  white_matter


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
1,-4254.501484,-3146.375011,-13987.750402,-1278.937776,-21494.765675,-40.011261,-493.658024,-2022.765156,-1027.323102,-113.88249,...,1239.0,4855.0,490.0,7638.0,2697.0,203.0,791.0,392.0,44.0,7635.0
2,-8364.006428,-2233.530946,-7478.199009,-545.129452,-18173.194154,-6.55246,-832.762155,-2242.633617,-580.754874,-48.528816,...,847.0,2060.0,198.0,5643.0,1175.0,306.0,794.0,227.0,16.0,5640.0
3,-750.918633,-1537.870293,-5109.884452,-367.208708,-7449.924526,-4.878979,-710.403726,-788.900062,-717.816508,-52.316408,...,621.0,1688.0,149.0,2607.0,860.0,273.0,315.0,281.0,19.0,2606.0
4,-23243.05533,-2150.33519,-11114.122589,-819.174219,-36712.648829,-7.828806,-837.672784,-1109.320938,-1040.380653,-30.280645,...,876.0,3308.0,332.0,9080.0,2632.0,336.0,448.0,427.0,12.0,9073.0
5,-398.564332,-1182.782911,-2434.603417,-224.105414,-4085.669061,-10.362631,-327.566281,-1252.994909,-411.695313,-31.759324,...,480.0,929.0,93.0,1594.0,374.0,131.0,493.0,165.0,12.0,1591.0
6,-610.920482,-1331.01154,-1598.945147,-263.352201,-3608.082557,-0.0,-353.922245,-781.480277,-888.124459,-22.364687,...,542.0,597.0,100.0,1381.0,250.0,144.0,307.0,354.0,9.0,1380.0
7,-71.9676,-1429.942974,-2233.800465,-407.271729,-3752.61893,-2.674281,-397.103045,-1301.301327,-358.977022,-9.648852,...,553.0,785.0,159.0,1374.0,304.0,155.0,463.0,147.0,4.0,1373.0
8,-181.838162,-1617.436439,-2268.35611,-184.822463,-4088.501436,-2.718192,-217.05954,-1694.042164,-736.166996,-14.883073,...,639.0,817.0,78.0,1537.0,187.0,91.0,632.0,273.0,6.0,1535.0
9,-215.179102,-1488.479292,-1590.068783,-577.543516,-3309.457694,-17.810289,-732.578198,-780.432455,-442.132234,-18.397677,...,610.0,588.0,223.0,1286.0,278.0,293.0,304.0,179.0,8.0,1284.0
11,-2117.894038,-2590.708846,-7146.935787,-1674.295699,-11978.072154,-6.383634,-344.197282,-1524.902403,-746.312588,-38.395573,...,999.0,2218.0,620.0,3880.0,1105.0,133.0,588.0,268.0,14.0,3876.0


------------- 

Dataframe:  grey_matter


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
1,-6168.55733,-21595.015867,-3369.967348,-7031.783502,-31048.117674,-167.49219,-3437.53358,-8073.420813,-3482.283378,-351.225592,...,7815.0,1163.0,2504.0,11109.0,2223.0,1246.0,2951.0,1281.0,126.0,11095.0
2,-44420.488471,-30687.737015,-8722.373643,-7451.727038,-82301.817159,-77.915536,-7948.192454,-10028.836586,-5748.908601,-549.70974,...,10674.0,2461.0,2534.0,23859.0,3540.0,2711.0,3520.0,2109.0,204.0,23846.0
3,-4866.267993,-18764.292562,-3788.333235,-3513.403095,-27390.414654,-140.467294,-4797.430688,-5595.713669,-5484.544584,-325.653698,...,7026.0,1370.0,1315.0,10138.0,1958.0,1769.0,2089.0,2087.0,126.0,10130.0
4,-44285.073467,-25474.342906,-1437.443428,-4286.093716,-69311.697098,-63.771461,-5375.35279,-9755.93515,-6872.455387,-272.864455,...,9339.0,536.0,1600.0,20725.0,1994.0,1870.0,3598.0,2573.0,108.0,20720.0
5,-1035.585174,-24222.360371,-2857.365305,-4219.820334,-28282.88123,-382.436676,-2856.36771,-12322.264379,-5566.970593,-471.006837,...,9124.0,1082.0,1643.0,10675.0,1983.0,1060.0,4604.0,2100.0,183.0,10671.0
6,-293.70078,-15371.144963,-1633.504534,-2841.712736,-17418.629775,-26.699037,-1949.391047,-5297.238885,-5699.335558,-146.144345,...,5812.0,595.0,1074.0,6572.0,934.0,756.0,2014.0,2135.0,58.0,6559.0
7,-1008.527986,-26068.414192,-8656.147186,-8934.517598,-35894.615531,-507.09826,-3014.00789,-10960.326897,-3241.697425,-803.442326,...,9479.0,2642.0,3204.0,12559.0,3123.0,1131.0,3947.0,1238.0,295.0,12547.0
8,-368.494984,-16166.912115,-2740.390568,-3074.57748,-19481.037448,-368.20314,-1515.190544,-9385.483552,-2638.694379,-294.129326,...,6045.0,983.0,1185.0,7253.0,1380.0,589.0,3424.0,1010.0,114.0,7250.0
9,-1085.930651,-16162.564607,-1795.891059,-7485.794101,-19235.695745,-325.434047,-2062.753556,-3918.80053,-2849.629441,-350.72448,...,5926.0,657.0,2607.0,7066.0,2063.0,794.0,1488.0,1100.0,131.0,7056.0
11,-18353.335729,-22604.058141,-11271.086018,-2751.810052,-50902.123877,-141.052333,-8520.778154,-5332.645851,-7662.688346,-71.174972,...,7896.0,2966.0,965.0,16011.0,2794.0,2965.0,1888.0,2651.0,29.0,15994.0


------------- 



Save the Results

In [44]:
import os
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from tqdm import tqdm

def save_csv_to_bids(dataframes_dict, bids_base_dir, analysis='atrophy_results', ses=None, dry_run=True):
    """
    Saves csv to a BIDS directory structure.
    
    Parameters:
    - dataframes_dict (dict): Dictionary containing dataframes with NIFTI data.
    - bids_base_dir (str): The base directory where the BIDS structure starts.
    - ses (str, optional): Session identifier. If None, defaults to '01'.
    
    Note:
    This function assumes a predefined BIDS directory structure and saves the CSV 
    accordingly.
    """
    
    for key, value in tqdm(dataframes_dict.items()):            
        # Define BIDS Directory Architecture
        sub_no = 'all'
        if ses is None:
            ses_no = '01'
        else:
            ses_no = ses
        
        # Define and Initialize the Save Directory
        out_dir = os.path.join(bids_base_dir, 'neuroimaging_analyses', f'ses-{ses_no}', f'sub-{sub_no}', analysis)
        os.makedirs(out_dir, exist_ok=True)
        
        # Save Image to BIDS Directory
        if dry_run:
            print(out_dir+f'/{key}.csv')
        else:
            value.to_csv(out_dir+f'/{key}.csv')
            print('Saved to: ', out_dir+f'/{key}.csv')

In [45]:
analysis='thresholded_atrophy_results'

In [46]:
save_csv_to_bids(sorted_damage_df_dict, bids_base_dir=base_directory, analysis=analysis, ses=None, dry_run=False)

100%|██████████| 3/3 [00:00<00:00, 606.55it/s]

Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/neuroimaging_analyses/ses-01/sub-all/thresholded_atrophy_results/cerebrospinal_fluid.csv
Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/neuroimaging_analyses/ses-01/sub-all/thresholded_atrophy_results/white_matter.csv
Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/neuroimaging_analyses/ses-01/sub-all/thresholded_atrophy_results/grey_matter.csv


All done. Enjoy your analyses. 

--Calvin 